### importing packages

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import re
import shutil
import bz2
import pickle
pd.options.display.max_columns = 200

### Read data in chunks

In [2]:
## loading the video extract
video = pd.read_csv(r"C:\Users\ujjwa\Downloads\Independent Study - Python Codes\video_file.csv")
video.rename(columns = {'timestampmsec': 'time', 'videonum':'video_id'}, inplace=True)

In [ ]:
## Loading data in chunks
os.chdir(r"C:\Ujjwal Documents\Courses Fall 2018\Independent Study\Final Independent Study Docs\Kickstarter Project\Video_Audio files\video_audio_merged_final")
chunk = 1000000
#path = r'C:\Users\Ujjwal Singh\Desktop\Pandas\Customers_10000.csv'
chunk_data = pd.read_csv('audio_video_output_cmd.csv', encoding='ISO-8859-1', chunksize=chunk, low_memory=False)
#data = next(chunk_data)

## Aggregating all the chunks of data
for chunks in range(0,1000):
    if(chunks==0):
        data = next(chunk_data)
    else:
        try:
            data = data.append(next(chunk_data))
        except StopIteration:
            break

### Merge LVA audio file

In [ ]:
## identify folders with different files
os.chdir(r"C:\Ujjwal Documents\Courses Fall 2018\Independent Study\ffmpeg")
## Get the list of folders
i=6
dir_list = []
for d in glob.glob('analysis*'):
    i+=1
    if(i<9):
        dir_list.append(d)
        print(dir_list)

In [ ]:
mainDF=pd.DataFrame()
for i in range(0,len(dir_list)):
    os.chdir(r"C:\Ujjwal Documents\Courses Fall 2018\Independent Study\ffmpeg")
    os.chdir(os.getcwd()+"\\"+dir_list[i])
    print(os.getcwd())
    for file in (glob.glob('*PARMS.csv')):
        data = pd.read_csv(file,delimiter=';')
        mainDF = pd.concat([mainDF,data], sort=False)

In [ ]:
## Extracting audio from audio file name.
z = re.sub(r'[^0-9]', '', r"C:\Ujjwal Documents\Courses Fall 2018\Independent Study\ffmpeg\analysis 1\video-223946-h264_high_(new).wav")
z[5:11]

## Function to extract ID from file path
def clean(x):
    y = re.sub(r'[^0-9]', '', 'r'+x)
    return(y[5:11])

In [ ]:
## cleaning column names and remove spaces.
for i in audio.columns:
    x = re.sub(r'[^A-Za-z0-9]','', i)
    audio.rename(columns = {i: x}, inplace=True)

In [ ]:
## splitting the audio file
id = []
for i in audio.audio_id.unique():
    id.append(i)
    
for j in id:
    audio_split = audio.loc[audio.audio_id == j]
    audio_split.to_csv(str(j)+'.csv')
    
os.chdir(r"C:\Users\Ujjwal Singh\Desktop\Pandas\audio_split)
for file in (glob.glob('*.csv')):
    data = pd.read_csv(file,delimiter=';')

### Missing files in audio folder

In [ ]:
## find missing files in audio and converting to right format of audio_split
os.chdir(r"C:\Ujjwal Documents\Courses Fall 2018\Independent Study\ffmpeg\missing_audios\audio")
for file in (glob.glob('*PARMS.csv')):
    data = pd.read_csv(file,delimiter=';')
    os.chdir(r"C:\Ujjwal Documents\Courses Fall 2018\Independent Study\ffmpeg\missing_audios\finalist")
    #cleaning column names and spaces
    for i in data.columns:
        #print(i)
        x = re.sub(r'[^A-Za-z0-9]','', i)
        data.rename(columns = {i: x}, inplace=True)
    data.rename(columns = {'StartPosSec' :'start', 'EndPosSec' : 'end'}, inplace=True)
    id = re.sub(r'[^0-9]', '', file)[:6]
    data['audio_id'] = str(id)
    data.drop(columns=[''], axis=1, inplace=True)
    data.to_csv(str(id)+'.csv')
    os.chdir(r"C:\Ujjwal Documents\Courses Fall 2018\Independent Study\ffmpeg\missing_audios\audio")

In [ ]:
x=[]
os.chdir(r"C:\Users\Ujjwal Singh\Desktop\Pandas\audio_split")
for file in (glob.glob('*.csv')):
    x.append(int(file[0:6]))
    
mis=[]
for i in video.video_id.unique():
    if i not in x:
        mis.append(i)
    else:
        pass

### merging audio and video data based on conditions - 1 & 2

In [ ]:
#merging audio and video
finalDF = pd.DataFrame()
for index,row in video.iterrows():
    #if (index>5665555): #last index of file
        video_df = pd.DataFrame()
        video_df = video_df.append(row)
        os.chdir(r'C:\Users\ujjwa\Downloads\Independent Study - Python Codes\audio files')
        for file in (glob.glob(str(row.video_id)+'.csv')):
            audio = pd.read_csv(file)
            audio.drop(columns = ['Unnamed: 0'], axis = 1, inplace=True)
            audio_df = audio.loc[((audio.start<=row.time) & (audio.end>=row.time))]
            finalDF = finalDF.append(pd.merge(video_df,audio_df, left_on='video_id', right_on='audio_id', how='left'))
        if (len(finalDF) == len(video.loc[video.video_id == row.video_id])):
            os.chdir(r'C:\Users\ujjwa\Downloads\Independent Study - Python Codes\old tech')
            finalDF.to_csv(str(index)+'_'+str(row.video_id)+'.csv')
            finalDF = pd.DataFrame()

In [ ]:
#merging audio and video
#miss = [560156]
for id in miss:
    finalDF = pd.DataFrame()
    os.chdir(r'C:\Users\ujjwa\Downloads\Independent Study - Python Codes\video split')
    video = pd.read_csv(glob.glob(str(id)+'.csv')[0])
    video.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace=True)
    os.chdir(r'C:\Users\ujjwa\Downloads\Independent Study - Python Codes\audio files')
    for file in (glob.glob(str(id)+'.csv')):
        audio = pd.read_csv(file)
        audio.drop(columns = ['Unnamed: 0'], axis = 1, inplace=True)
        for index,row in video.iterrows():
            video_df = pd.DataFrame()
            video_df = video_df.append(row)
            audio_df = audio.loc[((audio.start<=row.time) & (audio.end>=row.time))]
            finalDF = finalDF.append(pd.merge(video_df,audio_df, left_on='video_id', right_on='audio_id', how='left'))
        if(len(finalDF) == len(video)):
            os.chdir(r'C:\Users\ujjwa\Downloads\Independent Study - Python Codes\miss id')
            finalDF.to_csv(str(id)+'.csv')
            finalDF = pd.DataFrame()

### Final code for concatenating split files

In [ ]:
## Concatenate all split files generated into one single file.
output = pd.DataFrame()
file_missed = []
os.chdir(r"C:\Users\Ujjwal Singh\Desktop\Pandas\video_audio_merge")
for file in (glob.glob('*.csv')):
    content = pd.read_csv(file, delimiter=',', low_memory=False)
    #if any file contains a column 'Unnamed', find the index and drop before merging
    if content.columns.str.contains('Unnamed').sum() >= 1:
        #return index of True
        check = [i for i,x in enumerate(list(content.columns.str.contains('Unnamed'))) if x]
        #returns columns values to be dropped.
        col = content.columns[check]
        content.drop(columns=col, axis=1, inplace=True)
    if len(content.columns) == 113:
            output = output.append(content, sort='None')
    else:
        file_missed.append(file)
        pass